In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform

np.random.seed(0)

## `pdist` and `squareform` demo

In [2]:
pos1 = [(0,0),(0,1)]
pos2 = [(0,0), (0,1), (0,2)]
pos3 = [(0,0), (0,1), (0,2), (0,3)]

In [3]:
pdist(pos1)

array([1.])

In [4]:
pdist(pos2)

array([1., 2., 1.])

In [5]:
pdist(pos3)

array([1., 2., 3., 1., 2., 1.])

In [6]:
squareform(pdist(pos1))

array([[0., 1.],
       [1., 0.]])

In [7]:
squareform(pdist(pos2))

array([[0., 1., 2.],
       [1., 0., 1.],
       [2., 1., 0.]])

In [8]:
squareform(pdist(pos3))

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

## `np.where` demo

In [9]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [10]:
np.where(a > 6)

(array([7, 8, 9], dtype=int64),)

In [11]:
np.where(a > 6, a, 0)

array([0, 0, 0, 0, 0, 0, 0, 7, 8, 9])

In [12]:
b = squareform(pdist(pos3))
b

array([[0., 1., 2., 3.],
       [1., 0., 1., 2.],
       [2., 1., 0., 1.],
       [3., 2., 1., 0.]])

In [13]:
np.where(b < 2)

(array([0, 0, 1, 1, 1, 2, 2, 2, 3, 3], dtype=int64),
 array([0, 1, 0, 1, 2, 1, 2, 3, 2, 3], dtype=int64))

In [14]:
np.where(b < 2, b, None)

array([[0.0, 1.0, None, None],
       [1.0, 0.0, 1.0, None],
       [None, 1.0, 0.0, 1.0],
       [None, None, 1.0, 0.0]], dtype=object)

## Demo of numpy2d.py `MDSimulation.advance()`

In [15]:
pos = np.random.rand(1000,2)

In [16]:
dist = squareform(pdist(pos))
dist

array([[0.        , 0.1786471 , 0.14306129, ..., 0.15442332, 0.29151294,
        0.23362597],
       [0.1786471 , 0.        , 0.20562852, ..., 0.31012622, 0.13499124,
        0.05956292],
       [0.14306129, 0.20562852, 0.        , ..., 0.14567924, 0.34061286,
        0.26317858],
       ...,
       [0.15442332, 0.31012622, 0.14567924, ..., 0.        , 0.43695575,
        0.36939279],
       [0.29151294, 0.13499124, 0.34061286, ..., 0.43695575, 0.        ,
        0.08031155],
       [0.23362597, 0.05956292, 0.26317858, ..., 0.36939279, 0.08031155,
        0.        ]])

In [17]:
2* (2e-10*5.e6)

0.002

In [18]:
x,y = np.where(dist < 2* (2e-10*5e6))
x, y

(array([  0,   1,   2, ..., 997, 998, 999], dtype=int64),
 array([  0,   1,   2, ..., 997, 998, 999], dtype=int64))

`np.where` returns 2 arrays, the first one contains the row coordinates, and the 2nd contains the column coordinates

In [19]:
result = zip(x,y)
list(result)[0:7]

[(0, 0), (1, 1), (2, 2), (3, 3), (3, 71), (4, 4), (5, 5)]

We can see a ton of identical row-column coordinate pairs like (0,0), (1,1), or (273,273) because the values of dist on the diagonal of the matrix are all 0. This is because the distance of a particle from itself will always be 0.

Here in this small subsection of the result, there is one coordinate (3, 71). This means that the 3rd row and 71st column, i.e. `a[3][71]` has a value less than 2r.

The way squareform works, all collisions to the left of the diagonal will also exist on the right of the diagonal. This filtering by `x < y` only keeps elements above / to the right of the diagonal.

In [20]:
k = x < y
k

array([False, False, False, ..., False, False, False])

In [21]:
list(zip(x[k], y[k]))

[(3, 71), (54, 504), (172, 823), (190, 380), (321, 666), (618, 667)]

From the code above, we can see that while `(3,71)` is kept through the filtering, and identical value does exist at `(71, 3)` that was removed

In [22]:
# Number of particles.
n = 1000
# Scaling factor for distance, m-1. The box dimension is therefore 1/rscale.
rscale = 5.e6
# Use the van der Waals radius of Ar, about 0.2 nm.
r = 2e-10 * rscale
# Scale time by this factor, in s-1.
tscale = 1e9    # i.e. time will be measured in nanoseconds.
# Take the mean speed to be the root-mean-square velocity of Ar at 300 K.
sbar = 353 * rscale / tscale
# Time step in scaled time units.
FPS = 30
dt = 1/FPS
# Particle masses, scaled by some factor we're not using yet.
m = 1

# Initialize the particles' positions randomly.
# pos = np.random.random((n, 2))
# Initialize the particles velocities with random orientations and random
# magnitudes  around the mean speed, sbar.
theta = np.random.random(n) * 2 * np.pi
s0 = sbar * np.random.random(n)
vel = (s0 * np.array((np.cos(theta), np.sin(theta)))).T

In [28]:
for i,j in zip(x[k], y[k]):
    pos_i, vel_i = pos[i], vel[i]
    pos_j, vel_j = pos[j], vel[j]
    rel_pos, rel_vel = pos_i - pos_j, vel_i - vel_j
    print(rel_pos, rel_vel)

    r_rel = rel_pos @ rel_pos
    v_rel = rel_vel @ rel_pos

    print(r_rel, v_rel)

[ 0.00172229 -0.00015035] [0.96034577 1.61528445]
2.9888754512477223e-06 0.001411125216226672
[-0.00133198 -0.00092668] [-1.04874552 -1.04098726]
2.632913725970005e-06 0.002361574489716227
[ 0.00084272 -0.00103637] [0.34674391 1.48346203]
1.7842425663202045e-06 -0.0012452086368285441
[-0.00105482 -0.00127971] [-0.09059141 -0.84380713]
2.7503209380614794e-06 0.0011753895525026346
[ 0.00098136 -0.00064684] [0.7691553  0.78311605]
1.3814712316055442e-06 0.0002482649248253647
[-0.00167429  0.00080862] [ 1.35924558 -1.40770707]
3.45712633890071e-06 -0.0034140776920543276
